In [4]:
"""Swiss Metro Frequency changes by travel purpose -- Base case -- leisure

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',-0.215775,-20,20,0,'ASC_CAR')
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1,'ASC_SBB')
ASC_SM	 = Beta('ASC_SM',0.122635,-20,20,0,'ASC_SM')
B_CAR_COST	 = Beta('B_CAR_COST',-0.007514,-10,10,0,'B_CAR_COST')
B_HE	 = Beta('B_HE',-0.005887,-10,10,0,'B_HE')
B_SM_COST	 = Beta('B_SM_COST',-0.011591,-10,10,0,'B_SM_COST')
B_CAR_TIME	 = Beta('B_CAR_TIME',-0.013063,-10,10,0,'B_CAR_TIME')
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',-0.008814,-10,10,0,'B_TRAIN_TIME')
B_SM_TIME	 = Beta('B_SM_TIME',-0.010993,-10,10,0,'B_SM_TIME')
B_TRAIN_COST	 = Beta('B_TRAIN_COST',-0.029349,-10,10,0,'B_TRAIN_COST')
B_SENIOR	 = Beta('B_SENIOR',-1.864569,-10,10,0,'B_SENIOR')
B_GA	 = Beta('B_GA',0.518405,-10,80,0,'B_GA')
B_FIRST	 = Beta('B_FIRST',0.261506,-10,80,0,'B_FIRST')

# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
SBB_SP = ASC_SBB * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST
SM_SP = ASC_SM * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Exclude
# Here we use the "biogeme" way for backward compatibility
# Travelers are excluded with PURPOSE values 1, 2, 3, 5, 6, 7, 9.
exclude = ((  CHOICE   ==  0  ) \
           + (  PURPOSE == 1  ) + (  PURPOSE == 2  ) + (  PURPOSE == 3  ) + (  PURPOSE == 5  ) \
           + (  PURPOSE == 6  ) + (  PURPOSE == 7  ) + (  PURPOSE == 9  ) + (  AGE == 6  ))>0
database.remove(exclude)

#No need for estimating the model (it is already estimated). We want to obtain the individual probabilities and the market shares
prob1 = models.logit(V,av,1)
prob2 = models.logit(V,av,2)
prob3 = models.logit(V,av,3)
#Instead of reporting the choice in the simulation file, the probability of the chosen can be printed

#Simulation output
simulate = {'choice': CHOICE,
            '01 Prob. Train': prob1,
            '02 Prob. SM': prob2,
            '03 Prob. Car': prob3}

biogeme  = bio.BIOGEME(database,simulate)
biogeme.modelName = "MNL_SM_headway_base_leisure"
results = biogeme.simulate()
print("Results=",results.describe())

07-04-2019:21:16:33,584 WARNING  [biogeme.py:143] The chosen alternative is not available for the following observations (rownumber[choice]): 3564[3]-3565[3]-3566[3]-3567[3]-3568[3]-3569[3]-3570[3]-3571[3]-3572[3]-3573[3]-3574[3]-3575[3]-3576[3]-3577[3]-3578[3]-3579[3]-3580[3]-3581[3]-3582[3]-3583[3]-3584[3]-3585[3]-3586[3]-3587[3]-3588[3]-3589[3]-3590[3]


Results=             choice  01 Prob. Train  02 Prob. SM  03 Prob. Car
count  2313.000000    2.313000e+03  2313.000000   2313.000000
mean      2.435365    1.107616e-01     0.561294      0.327945
std       0.600817    1.280451e-01     0.185565      0.191222
min       1.000000    7.740791e-07     0.000533      0.000000
25%       2.000000    3.384286e-02     0.433812      0.189136
50%       2.000000    7.096902e-02     0.565455      0.301721
75%       3.000000    1.352010e-01     0.696753      0.450520
max       3.000000    8.044601e-01     0.967307      0.999458
